## Env Setup

In [1]:
!pip install mne

In [2]:
import os
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import numpy as np

import mne
from mne.datasets import eegbci
mne.__version__

'0.23.0'

## Get Data

In [3]:
data_dir = "data"
subjects = [1] # 1 indexed
#runs = [5, 9, 13] # hands versus foots imaginary
runs = [6, 10, 14] # hands versus foots real
#runs = [3, 7, 11] # left vs right fist imaginary
#runs = [4, 8, 12] # left vs right fist real


raw_fnames = {}
for i, d in enumerate(os.listdir(data_dir)):
    if os.path.isdir(os.path.join(data_dir, d)) and i + 1 in subjects:
        raw_fnames[d] = os.listdir(os.path.join(data_dir, d))

dataset = []
sfreq = None
for d in raw_fnames:
    subject = []
    b = False
    for i, f in enumerate(raw_fnames[d]):
        if f.endswith(".edf") and int(f.split('R')[1].split(".")[0]) in runs:
            subject_data = mne.io.read_raw_edf(os.path.join(data_dir, d, f), preload=True)
            if sfreq == None:
                sfreq = subject_data.info["sfreq"]
            if subject_data.info["sfreq"] == sfreq:
                subject.append(subject_data)
            else:
                b = True
                break
    if b:
        continue
    dataset.append(mne.concatenate_raws(subject))
dataset = mne.concatenate_raws(dataset)

Extracting EDF parameters from C:\Users\Rock_\Desktop\Projects\Total_perspective_vortex\data\S001\S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\Rock_\Desktop\Projects\Total_perspective_vortex\data\S001\S001R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\Rock_\Desktop\Projects\Total_perspective_vortex\data\S001\S001R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...


In [4]:
print(dataset)
print(dataset.info)
print(dataset.info["ch_names"])
# events
print(dataset.annotations)

<RawEDF | S001R06.edf, 64 x 60000 (375.0 s), ~29.4 MB, data loaded>
<Info | 7 non-empty values
 bads: []
 ch_names: Fc5., Fc3., Fc1., Fcz., Fc2., Fc4., Fc6., C5.., C3.., C1.., ...
 chs: 64 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 80.0 Hz
 meas_date: 2009-08-12 16:15:00 UTC
 nchan: 64
 projs: []
 sfreq: 160.0 Hz
>
['Fc5.', 'Fc3.', 'Fc1.', 'Fcz.', 'Fc2.', 'Fc4.', 'Fc6.', 'C5..', 'C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'C6..', 'Cp5.', 'Cp3.', 'Cp1.', 'Cpz.', 'Cp2.', 'Cp4.', 'Cp6.', 'Fp1.', 'Fpz.', 'Fp2.', 'Af7.', 'Af3.', 'Afz.', 'Af4.', 'Af8.', 'F7..', 'F5..', 'F3..', 'F1..', 'Fz..', 'F2..', 'F4..', 'F6..', 'F8..', 'Ft7.', 'Ft8.', 'T7..', 'T8..', 'T9..', 'T10.', 'Tp7.', 'Tp8.', 'P7..', 'P5..', 'P3..', 'P1..', 'Pz..', 'P2..', 'P4..', 'P6..', 'P8..', 'Po7.', 'Po3.', 'Poz.', 'Po4.', 'Po8.', 'O1..', 'Oz..', 'O2..', 'Iz..']
<Annotations | 94 segments: BAD boundary (2), EDGE boundary (2), T0 (45), ...>


In [5]:
# remove dots from channel's names
dataset = dataset.rename_channels(lambda s: s.strip("."))

In [6]:
# set montage
montage = mne.channels.make_standard_montage("standard_1020")

eegbci.standardize(dataset)  # set channel names
dataset.set_montage(montage)

<RawEDF | S001R06.edf, 64 x 60000 (375.0 s), ~29.4 MB, data loaded>

In [7]:
montage = dataset.get_montage()
p = montage.plot()

Creating RawArray with float64 data, n_channels=64, n_times=1
    Range : 0 ... 0 =      0.000 ...     0.000 secs
Ready.


In [8]:
# plot data
p = mne.viz.plot_raw(dataset, scalings={"eeg": 75e-6})

## Filter

In [9]:
# FirWin filter
dataset_tmp = dataset.copy()
# set montage again since its is not copyed
dataset_tmp.set_montage(montage)
dataset_tmp.filter(7, 30, fir_design='firwin', skip_by_annotation='edge')
filtered_dataset = dataset_tmp

Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 265 samples (1.656 sec)



In [10]:
# plot data
p = mne.viz.plot_raw(filtered_dataset, scalings={"eeg": 75e-6})

## Set Labels

In [11]:
event_id = dict(T1=0, T2=1)
# avoid classification of evoked responses by using epochs that start 1s after cue onset.
tmin, tmax = -1., 4.

events, _ = mne.events_from_annotations(filtered_dataset, event_id=event_id)
picks = mne.pick_types(filtered_dataset.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')
epochs = mne.Epochs(filtered_dataset, events, event_id, tmin, tmax, proj=True, picks=picks, baseline=None, preload=True)   
labels = epochs.events[:, -1]

epochs.get_data().shape, labels

Used Annotations descriptions: ['T1', 'T2']
Not setting metadata
Not setting metadata
45 matching events found
No baseline correction applied
0 projection items activated
Loading data for 45 events and 801 original time points ...
0 bad epochs dropped


((45, 64, 801),
 array([1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0,
        1]))

## Pipelines

In [12]:
from mne.decoding import CSP

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ShuffleSplit, StratifiedKFold, cross_val_score
from mne.decoding import UnsupervisedSpatialFilter, Vectorizer, Scaler

https://mne.tools/stable/auto_examples/decoding/decoding_csp_eeg.html#sphx-glr-auto-examples-decoding-decoding-csp-eeg-py

Subject 1, runs 6, 10, 14

In [13]:
epochs_data = epochs.get_data()

print(epochs_data.shape, labels.shape)

n_splits = 5  # how many folds to use for cross-validation
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
#cv = ShuffleSplit(10, test_size=0.2, random_state=42)

# Assemble a classifier
lda = LinearDiscriminantAnalysis()
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
sca = Scaler(epochs.info)

# Use scikit-learn Pipeline with cross_val_score function
clf = Pipeline([('SCA', sca), ('CSP', csp), ('LDA', lda)])

scores = cross_val_score(clf, epochs_data, labels, cv=cv, n_jobs=1)
clf = clf.fit(epochs_data, labels)

# plot CSP patterns estimated on full data for visualization
p = csp.plot_patterns(epochs.info, ch_type='eeg', units='Patterns (AU)', size=1.5)

# Printing the results
class_balance = np.mean(labels == 0)
class_balance = max(class_balance, 1. - class_balance)
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance))

(45, 64, 801) (45,)
Computing rank from data with rank=None
    Using tolerance 2.2e+02 (2.2e-16 eps * 64 dim * 1.6e+16  max singular value)
    Estimated rank (mag): 64
    MAG: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 2.3e+02 (2.2e-16 eps * 64 dim * 1.6e+16  max singular value)
    Estimated rank (mag): 64
    MAG: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 2.1e+02 (2.2e-16 eps * 64 dim * 1.5e+16  max singular value)
    Estimated rank (mag): 64
    MAG: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 2.4e+02 (2.2e-16 eps * 64 dim * 1.7e+16  

## Running Classification

In [14]:
print("X shape= ", epochs_data.shape, "y shape= ", labels.shape)

scores = []
for n in range(epochs_data.shape[0]):
    pred = clf.predict(epochs_data[n:n + 1, :, :])
    print("n=", n, "pred= ", pred, "truth= ", labels[n:n + 1])
    scores.append(1 - np.abs(pred[0] - labels[n:n + 1][0]))
print("Mean acc= ", np.mean(scores))

X shape=  (45, 64, 801) y shape=  (45,)
n= 0 pred=  [1] truth=  [1]
n= 1 pred=  [0] truth=  [0]
n= 2 pred=  [0] truth=  [0]
n= 3 pred=  [1] truth=  [1]
n= 4 pred=  [0] truth=  [0]
n= 5 pred=  [1] truth=  [1]
n= 6 pred=  [1] truth=  [1]
n= 7 pred=  [0] truth=  [0]
n= 8 pred=  [0] truth=  [0]
n= 9 pred=  [1] truth=  [1]
n= 10 pred=  [1] truth=  [1]
n= 11 pred=  [0] truth=  [0]
n= 12 pred=  [0] truth=  [0]
n= 13 pred=  [1] truth=  [1]
n= 14 pred=  [1] truth=  [1]
n= 15 pred=  [0] truth=  [0]
n= 16 pred=  [1] truth=  [1]
n= 17 pred=  [1] truth=  [1]
n= 18 pred=  [0] truth=  [0]
n= 19 pred=  [1] truth=  [1]
n= 20 pred=  [0] truth=  [0]
n= 21 pred=  [0] truth=  [0]
n= 22 pred=  [1] truth=  [1]
n= 23 pred=  [1] truth=  [1]
n= 24 pred=  [0] truth=  [0]
n= 25 pred=  [0] truth=  [0]
n= 26 pred=  [1] truth=  [1]
n= 27 pred=  [1] truth=  [1]
n= 28 pred=  [0] truth=  [0]
n= 29 pred=  [1] truth=  [1]
n= 30 pred=  [1] truth=  [1]
n= 31 pred=  [0] truth=  [0]
n= 32 pred=  [1] truth=  [1]
n= 33 pred=  